# Bumuo ng isang regression model: ihanda at i-visualize ang data

## **Linear Regression para sa mga Kalabasa - Aralin 2**
#### Panimula

Ngayon na mayroon ka nang mga kasangkapan na kailangan mo upang simulan ang pagbuo ng mga modelo ng machine learning gamit ang Tidymodels at Tidyverse, handa ka nang magsimulang magtanong tungkol sa iyong data. Habang nagtatrabaho ka sa data at nag-aaplay ng mga solusyon sa ML, napakahalaga na maunawaan kung paano magtanong ng tamang tanong upang ma-unlock nang maayos ang potensyal ng iyong dataset.

Sa araling ito, matututuhan mo:

-   Paano ihanda ang iyong data para sa pagbuo ng modelo.

-   Paano gamitin ang `ggplot2` para sa pag-visualize ng data.

Ang tanong na nais mong masagot ang magpapasiya kung anong uri ng mga algorithm ng ML ang iyong gagamitin. At ang kalidad ng sagot na makukuha mo ay lubos na nakadepende sa kalikasan ng iyong data.

Tingnan natin ito sa pamamagitan ng isang praktikal na ehersisyo.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Likha ni @allison_horst</figcaption>


<!--![Likha ni \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.tl.jpg)<br>Likha ni \@allison_horst-->


## 1. Pag-import ng data ng pumpkins at pag-summon sa Tidyverse

Kakailanganin natin ang mga sumusunod na package para hatiin at suriin ang araling ito:

-   `tidyverse`: Ang [tidyverse](https://www.tidyverse.org/) ay isang [koleksyon ng mga R package](https://www.tidyverse.org/packages) na idinisenyo upang gawing mas mabilis, mas madali, at mas masaya ang data science!

Maaari mo itong mai-install gamit ang:

`install.packages(c("tidyverse"))`

Ang script sa ibaba ay sinusuri kung mayroon ka nang mga kinakailangang package para makumpleto ang module na ito at awtomatikong ini-install ang mga ito kung may kulang.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Ngayon, simulan natin ang ilang mga package at i-load ang [data](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) na ibinigay para sa araling ito!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Ang mabilis na `glimpse()` ay agad nagpapakita na may mga blangko at halo ng mga string (`chr`) at numerikong datos (`dbl`). Ang `Date` ay nasa uri ng character at mayroon ding kakaibang kolum na tinatawag na `Package` kung saan ang datos ay halo ng `sacks`, `bins`, at iba pang mga halaga. Sa totoo lang, medyo magulo ang datos 😤.

Sa katunayan, hindi karaniwan na makatanggap ng dataset na ganap na handa para gamitin sa paglikha ng ML model nang direkta. Pero huwag mag-alala, sa araling ito, matututuhan mo kung paano ihanda ang isang raw dataset gamit ang mga karaniwang R libraries 🧑‍🔧. Matututuhan mo rin ang iba't ibang teknik para ma-visualize ang datos.📈📊
<br>

> Paalala: Ang pipe operator (`%>%`) ay nagsasagawa ng mga operasyon sa lohikal na pagkakasunod-sunod sa pamamagitan ng pagpapasa ng isang object sa isang function o call expression. Maaari mong isipin ang pipe operator bilang nagsasabi ng "at pagkatapos" sa iyong code.


## 2. Suriin ang nawawalang datos

Isa sa mga pinakakaraniwang isyu na kailangang harapin ng mga data scientist ay ang hindi kumpleto o nawawalang datos. Ang R ay kumakatawan sa nawawala, o hindi kilalang mga halaga, gamit ang espesyal na sentinel value: `NA` (Not Available).

Paano natin malalaman kung ang data frame ay may nawawalang mga halaga?
<br>
-   Isang direktang paraan ay ang paggamit ng base R function na `anyNA` na nagbabalik ng mga logical na bagay na `TRUE` o `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Mukhang may kulang na datos! Magandang simula ito.

-   Isa pang paraan ay ang paggamit ng function na `is.na()` na nagpapakita kung aling mga elemento sa bawat column ang nawawala gamit ang lohikal na `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Okay, natapos na ang trabaho pero sa ganitong kalaking data frame, magiging hindi epektibo at halos imposibleng suriin ang lahat ng mga row at column nang paisa-isa😴.

-   Isang mas madaling paraan ay ang pag-compute ng kabuuan ng mga nawawalang halaga para sa bawat column:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Mas mabuti! May kulang na datos, pero baka hindi ito mahalaga para sa kasalukuyang gawain. Tingnan natin kung ano ang maidudulot ng karagdagang pagsusuri.

> Kasama ng mga kahanga-hangang set ng mga package at function, ang R ay may napakagandang dokumentasyon. Halimbawa, gamitin ang `help(colSums)` o `?colSums` para malaman ang higit pa tungkol sa function.


## 3. Dplyr: Isang Gramatika ng Manipulasyon ng Datos

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Likha ni @allison_horst</figcaption>


<!--![Likha ni \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.tl.png)<br/>Likha ni \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), isang package sa Tidyverse, ay isang gramatika ng pag-manipula ng datos na nagbibigay ng pare-parehong hanay ng mga pandiwa na tumutulong sa iyo na malutas ang mga karaniwang hamon sa pag-manipula ng datos. Sa seksyong ito, tatalakayin natin ang ilan sa mga pandiwa ng dplyr!  
<br>


#### dplyr::select()

Ang `select()` ay isang function sa package na `dplyr` na tumutulong sa iyo na pumili ng mga column na itatago o aalisin.

Upang gawing mas madali ang pagtrabaho sa iyong data frame, alisin ang ilang mga column nito gamit ang `select()`, at itago lamang ang mga column na kailangan mo.

Halimbawa, sa ehersisyong ito, ang ating pagsusuri ay maglalaman ng mga column na `Package`, `Low Price`, `High Price` at `Date`. Piliin natin ang mga column na ito.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

Ang `mutate()` ay isang function sa package na `dplyr` na tumutulong sa iyo na lumikha o magbago ng mga column, habang pinapanatili ang mga umiiral na column.

Ang pangkalahatang istruktura ng `mutate` ay:

`data %>%   mutate(new_column_name = what_it_contains)`

Subukan nating gamitin ang `mutate` gamit ang column na `Date` sa pamamagitan ng pagsasagawa ng mga sumusunod na operasyon:

1. I-convert ang mga petsa (na kasalukuyang nasa uri na character) sa format ng buwan (ang mga ito ay US dates, kaya ang format ay `MM/DD/YYYY`).

2. Kunin ang buwan mula sa mga petsa at ilagay ito sa isang bagong column.

Sa R, ang package na [lubridate](https://lubridate.tidyverse.org/) ay nagpapadali sa pagproseso ng Date-time data. Kaya, gamitin natin ang `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` at tingnan kung paano makakamit ang mga nabanggit na layunin. Maaari nating tanggalin ang column na `Date` dahil hindi na natin ito kakailanganin sa mga susunod na operasyon.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Ngayon, gumawa tayo ng bagong column na `Price`, na kumakatawan sa karaniwang presyo ng kalabasa. Kunin natin ang average ng mga column na `Low Price` at `High Price` para punan ang bagong column na Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

"Pero teka!", sasabihin mo pagkatapos mong silipin ang buong data set gamit ang `View(pumpkins)`, "May kakaiba dito!"🤔

Kung titingnan mo ang `Package` na kolum, ang mga kalabasa ay ibinebenta sa iba't ibang mga sukat. Ang ilan ay ibinebenta sa sukat na `1 1/9 bushel`, ang ilan naman sa `1/2 bushel`, ang iba ay per kalabasa, per pound, at ang iba naman ay nasa malalaking kahon na may iba't ibang lapad.

Suriin natin ito:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Ang galing!👏

Mukhang mahirap talagang timbangin nang pare-pareho ang mga kalabasa, kaya't mag-filter tayo sa pamamagitan ng pagpili lamang ng mga kalabasa na may string *bushel* sa `Package` column at ilagay ito sa bagong data frame na `new_pumpkins`.


#### dplyr::filter() at stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): gumagawa ng subset ng data na naglalaman lamang ng **mga row** na tumutugon sa iyong mga kondisyon, sa kasong ito, mga kalabasa na may string na *bushel* sa `Package` column.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): tumutukoy kung mayroong o wala ang isang pattern sa isang string.

Ang [`stringr`](https://github.com/tidyverse/stringr) package ay nagbibigay ng simpleng mga function para sa karaniwang mga operasyon sa string.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Makikita mo na napaliit na natin sa humigit-kumulang 415 na mga hilera ng datos na naglalaman ng mga kalabasa kada bushel.🤩


#### dplyr::case_when()

**Pero teka! May isa pang bagay na kailangang gawin**

Napansin mo ba na ang dami ng bushel ay nag-iiba sa bawat row? Kailangan mong gawing normal ang pagpepresyo upang maipakita ang presyo kada bushel, hindi kada 1 1/9 o 1/2 bushel. Panahon na para mag-compute upang ma-standardize ito.

Gagamitin natin ang function [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) upang *i-mutate* ang column na Price batay sa ilang kondisyon. Ang `case_when` ay nagbibigay-daan sa iyo na mag-vectorize ng maraming `if_else()` na pahayag.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Ngayon, maaari nating suriin ang presyo bawat yunit batay sa kanilang sukat sa bushel. Ang lahat ng pag-aaral na ito tungkol sa bushel ng kalabasa, gayunpaman, ay nagpapakita kung gaano kahalaga ang `maunawaan ang kalikasan ng iyong datos`!

> ✅ Ayon sa [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), ang timbang ng isang bushel ay nakadepende sa uri ng produkto, dahil ito ay isang sukat ng dami. "Ang isang bushel ng kamatis, halimbawa, ay dapat tumimbang ng 56 pounds... Ang mga dahon at gulay ay kumukuha ng mas maraming espasyo ngunit mas magaan, kaya ang isang bushel ng spinach ay 20 pounds lamang." Medyo komplikado ito! Huwag na nating alalahanin ang paggawa ng conversion mula bushel patungong pounds, at sa halip ay magpresyo batay sa bushel. Ang lahat ng pag-aaral na ito tungkol sa bushel ng kalabasa, gayunpaman, ay nagpapakita kung gaano kahalaga ang maunawaan ang kalikasan ng iyong datos!
>
> ✅ Napansin mo ba na ang mga kalabasa na ibinebenta sa kalahating bushel ay napakamahal? Kaya mo bang alamin kung bakit? Pahiwatig: ang maliliit na kalabasa ay mas mahal kaysa sa malalaki, marahil dahil mas marami ang mga ito bawat bushel, dahil sa espasyong hindi nagagamit na kinukuha ng isang malaking guwang na pie pumpkin.


Ngayon, sa huli, para sa kasiyahan ng pakikipagsapalaran 💁‍♀️, ilipat natin ang column na Buwan sa unang posisyon, ibig sabihin, `bago` ang column na `Package`.

Ginagamit ang `dplyr::relocate()` upang baguhin ang posisyon ng mga column.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Magaling!👌 Mayroon ka na ngayong malinis at maayos na dataset na maaari mong gamitin para bumuo ng bago mong regression model!


## 4. Pagpapakita ng Datos gamit ang ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infographic ni Dasani Madipalli</figcaption>


<!--![Infographic ni Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.tl.png){width="600"}-->

May isang *matalinong* kasabihan na ganito ang sinasabi:

> "Ang simpleng grapiko ay nagdala ng mas maraming impormasyon sa isipan ng data analyst kaysa sa anumang ibang kagamitan." --- John Tukey

Bahagi ng trabaho ng isang data scientist ang ipakita ang kalidad at katangian ng datos na kanilang pinagtatrabahuhan. Upang magawa ito, madalas silang gumagawa ng mga kawili-wiling visualisasyon, o mga plot, grapiko, at tsart, na nagpapakita ng iba't ibang aspeto ng datos. Sa ganitong paraan, nagagawa nilang ipakita nang biswal ang mga relasyon at puwang na mahirap matukoy sa ibang paraan.

Ang mga visualisasyon ay maaari ring makatulong sa pagtukoy ng pinakaangkop na teknik sa machine learning para sa datos. Halimbawa, ang isang scatterplot na tila sumusunod sa isang linya ay nagpapahiwatig na ang datos ay maaaring gamitin para sa linear regression.

Nag-aalok ang R ng ilang sistema para sa paggawa ng mga grapiko, ngunit ang [`ggplot2`](https://ggplot2.tidyverse.org/index.html) ay isa sa pinakaelegante at pinakamaraming gamit. Pinapayagan ka ng `ggplot2` na bumuo ng mga grapiko sa pamamagitan ng **pagsasama-sama ng mga independiyenteng bahagi**.

Magsimula tayo sa isang simpleng scatter plot para sa mga column na Price at Month.

Sa kasong ito, magsisimula tayo sa [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), magbibigay ng dataset at aesthetic mapping (gamit ang [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) at pagkatapos ay magdaragdag ng mga layer (tulad ng [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) para sa scatter plots.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Kapaki-pakinabang ba ang plot na ito 🤷? Mayroon bang anumang nakakagulat tungkol dito?

Hindi ito masyadong kapaki-pakinabang dahil ang tanging ginagawa nito ay ipakita ang iyong data bilang kalat ng mga puntos sa isang partikular na buwan.  
<br>


### **Paano natin ito magagamit nang maayos?**

Para makapagpakita ng kapaki-pakinabang na datos sa mga chart, kadalasan kailangan mong i-grupo ang datos sa isang paraan. Halimbawa, sa ating sitwasyon, ang pagkuha ng average na presyo ng kalabasa bawat buwan ay magbibigay ng mas malinaw na pananaw sa mga nakatagong pattern sa ating datos. Ito ay nagdadala sa atin sa isa pang **dplyr** na pamamaraan:

#### `dplyr::group_by() %>% summarize()`

Ang pinagsama-samang datos sa R ay madaling makalkula gamit ang

`dplyr::group_by() %>% summarize()`

-   Ang `dplyr::group_by()` ay binabago ang unit ng pagsusuri mula sa buong dataset patungo sa mga indibidwal na grupo tulad ng bawat buwan.

-   Ang `dplyr::summarize()` ay lumilikha ng bagong data frame na may isang column para sa bawat variable ng grupo at isang column para sa bawat istatistikang buod na iyong tinukoy.

Halimbawa, maaari nating gamitin ang `dplyr::group_by() %>% summarize()` upang i-grupo ang mga kalabasa batay sa **Month** na column at pagkatapos ay hanapin ang **mean price** para sa bawat buwan.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Maikli!✨

Ang mga kategoryang tampok tulad ng mga buwan ay mas mainam na ipakita gamit ang isang bar plot 📊. Ang mga layer na responsable para sa mga bar chart ay `geom_bar()` at `geom_col()`. Tingnan ang `?geom_bar` para malaman ang higit pa.

Gumawa tayo ng isa!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Mas kapaki-pakinabang ang visualisasyong ito ng datos! Mukhang ipinapakita nito na ang pinakamataas na presyo ng kalabasa ay nangyayari tuwing Setyembre at Oktubre. Tugma ba ito sa inaasahan mo? Bakit o bakit hindi?

Binabati kita sa pagtatapos ng ikalawang aralin 👏! Inihanda mo ang iyong datos para sa paggawa ng modelo, pagkatapos ay natuklasan ang mas maraming kaalaman gamit ang mga visualisasyon!



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang orihinal na wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
